# Iris Flower Classification with Scikit-Learn and Hopsworks

In this notebook we will, 

1. Load the Iris Flower dataset into Pandas from a CSV file
2. Save the features to a feature group
3. Create a feature view from the feature group
4. Read the train/test features and labels using the feature view
5. Train a KNN Model using SkLearn
6. Save the trained model to Hopsworks
7. Launch a serving instance to serve the trained model (KServe)
8. Send a prediction request to the served model
9. Start a Gradio UI to interactively make predictions using the input features for the Iris Model

## Import libraries

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib
import numpy as np
import time
import json
import random
import hopsworks
import pandas as pd
from sklearn import preprocessing

### Not app.hopsworks.ai ?

If you are running your own Hopsworks cluster (not app.hopsworks.ai):

 * uncomment the cell below
 * fill in details for your cluster
 * run the cel

In [ ]:
#key=""
#with open("api-key.txt", "r") as f:
#    key = f.read().rstrip()
#os.environ['HOPSWORKS_PROJECT']="cjsurf"
#os.environ['HOPSWORKS_HOST']="35.187.178.84"
#os.environ['HOPSWORKS_API_KEY']=key    

### Connect to your Hopsworks cluster

If you only set the HOPSWORKS_API_KEY, it will assume you are connecting to app.hopsworks.ai.
Set HOPSWORKS_HOST and HOPSWORKS_PROJECT environment variables to connect to a different Hopsworks cluster.

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

## Prepare Training Dataset

### Load Iris Dataset (csv)

In [ ]:
iris_df = pd.read_csv("https://repo.hops.works/master/hopsworks-tutorials/data/iris.csv")
iris_df.head()

In [ ]:
iris_df.info()

### Save Features to the Feature Store

We can save two feature groups (hive tables), one called `iris_features` that contains the iris features and the corresponding numeric label, and another feature group called `iris_labels_lookup` for converting the numeric iris label back to categorical.

**Note**: To be able to run the feature store code, you first have to enable the Feature Store Service in your project. To do this, go to the "Settings" tab in your project, select the feature store service and click "Save". 

In [ ]:
iris_fg = fs.create_feature_group(name="iris",
                                  version=1,
                                  primary_key=["sepal_length","sepal_width","petal_length","petal_width"],
                                  description="Iris flower dataset"
                                 )
iris_fg.insert(iris_df)

In [ ]:
query = iris_fg.select_all()
feature_view = fs.create_feature_view(name="iris",
                                      version=1,
                                      description="Read from Iris flower dataset",
                                      labels=["variety"],
                                      query=query)

In [ ]:
X_train, y_train, X_test, y_test = feature_view.train_test_split(0.2)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_encoded=le.fit_transform(y_train['variety'])
y_test_encoded=le.fit_transform(y_test['variety'])

model = KNeighborsClassifier(n_neighbors=4)
model.fit(X_train, y_train_encoded) 

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test_encoded, y_pred)

metrics = {
    "rmse" : rmse
}
print(metrics)

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import os

model_dir="iris_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)
# Put the pickled model and the predictor script in the 'iris_model' directory
# Then save the whole 'iris_model' directory to the model registry
pickle='knn_iris_model.pkl'

joblib.dump(model, model_dir + "/" + pickle)

input_example = X_train.sample()
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

iris_model = mr.python.create_model(
    version=1,
    name="iris_model", 
    metrics=metrics,
    model_schema=model_schema,
    input_example=input_example, 
    description="Iris Flower Predictor")

In [ ]:
%%writefile iris_model/iris_predictor.py

import joblib
import os

class Predict(object):
    
    def __init__(self):
        # NOTE: env var ARTIFACT_FILES_PATH has the local path to the model artifact files        
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/knn_iris_model.pkl")


    def predict(self, inputs):
        """ Serves a prediction request from a trained model"""
        return self.model.predict(inputs).tolist()

In [ ]:
# Save the whole 'iris_model' directory to the model registry, including the model and the predictor script
iris_model.save(model_dir)

In [ ]:
predictor_script_path = iris_model.version_path + "/iris_predictor.py"
irisclassifier = iris_model.deploy(name = "irisdeployed",
                                   script_file=predictor_script_path,  
                                   model_server="PYTHON", 
                                   serving_tool="KSERVE")
irisclassifier.describe()

In [ ]:
irisclassifier.start()

In [ ]:
irisclassifier.get_logs()

### Send Prediction Requests to the Deployed Model

For making inference requests you can use the utility method `.predict()` from the deployment object.

In [ ]:
input_list = list(iris_model.input_example)

data = {"instances" : [input_list]}
res = irisclassifier.predict(data)
print(input_list)
print(le.inverse_transform([res["predictions"][0]]))

## Try out your Model Interactively

We will build a user interface with Gradio to allow you to enter the 4 feature values (sepal length/width and petal length/width), producing a prediction of the type of iris flower.

First, we have to install the gradio library.

In [ ]:
!pip install gradio --quiet
!pip install typing-extensions==4.3.0

In [ ]:
import gradio as gr

def iris(sl, sw, pl, pw):
    list_inputs = []
    list_inputs.append(sl)
    list_inputs.append(sw)
    list_inputs.append(pl)
    list_inputs.append(pw)
    data = {
        "instances": [list_inputs]
    }
    res = irisclassifier.predict(data)
    # Convert the numerical representation of the label back to it's original iris flower name.
    return le.inverse_transform([res["predictions"][0]])[0]

demo = gr.Interface(
    fn=iris,
    title="Iris Flower Predictive Analytics",
    description="Experiment with sepal/petal lengths/widths to predict which flower it is.",
    allow_flagging="never",
    inputs=[
        gr.inputs.Number(default=1.0, label="sepal length (cm)"),
        gr.inputs.Number(default=1.0, label="sepal width (cm)"),
        gr.inputs.Number(default=1.0, label="petal length (cm)"),
        gr.inputs.Number(default=1.0, label="petal width (cm)"),
        ],
    outputs="text")

demo.launch(share=True)